In [2]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Load the MovieLens dataset
movies = pd.read_csv("../Labb/Big/movies.csv")
ratings = pd.read_csv("../Labb/Big/ratings.csv")

In [3]:
# Create a pivot table of user ratings for each movie
movie_ratings = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)

# Convert the pivot table to a sparse matrix
movie_ratings_sparse = csr_matrix(movie_ratings)


c:\Users\Danne\.virtualenvs\Machine-learning-PT8MuEHh\lib\site-packages\pandas\core\reshape\reshape.py:126: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [ ]:
# Create a KNN model with cosine similarity
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

# Fit the model to the sparse matrix of user ratings
knn.fit(movie_ratings_sparse)


In [ ]:
def get_movie_recommendations(movie_name, knn_model, movie_ratings_matrix, movies_df, n_recommendations=10):
    """
    Get movie recommendations based on a given movie name using a KNN model.
    
    Arguments:
    - movie_name: Name of the movie to get recommendations for
    - knn_model: KNN model to use for generating recommendations
    - movie_ratings_matrix: Sparse matrix of user ratings for each movie
    - movies_df: DataFrame of movie metadata
    - n_recommendations: Number of recommendations to generate (default=10)
    
    Returns:
    - List of recommended movie titles
    """
    # Get the movie ID for the given movie name
    movie_id = movies_df[movies_df['title'] == movie_name]['movieId'].iloc[0]
    
    # Get the indices of the most similar movies based on the KNN model
    movie_indices = knn_model.kneighbors(movie_ratings_matrix[movie_id], n_neighbors=n_recommendations+1)[1]
    
    # Get the titles of the recommended movies
    recommended_movies = []
    for i in range(1, len(movie_indices)):
        recommended_movie_title = movies_df[movies_df['movieId'] == movie_indices[i]]['title'].iloc[0]
        recommended_movies.append(recommended_movie_title)
    
    return recommended_movies


In [ ]:
# Test the recommender system by generating recommendations for a given movie
movie_name = 'Toy Story (1995)'
recommended_movies = get_movie_recommendations(movie_name, knn, movie_ratings_sparse, movies)
print(f"Recommended movies for '{movie_name}':")
for i, movie_title in enumerate(recommended_movies):
    print(f"{i+1}. {movie_title}")
